In [1]:

from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/Colab Notebooks/'


Mounted at /content/drive


In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [4]:
dhs_data1 = pd.read_stata(abspath_curr + '/data/PK_2019_DHS/PKHH7ADT/PKHH7AFL.DTA')
dhs_data1.head()
data_types1 = dhs_data1.dtypes
for var_name, data_type in data_types1.iteritems():
    print(f"Variable: {var_name}, Data Type: {data_type}")

Variable: qhclust, Data Type: int16
Variable: qhnumber, Data Type: int16
Variable: qhregion, Data Type: category
Variable: qhdist, Data Type: category
Variable: qhtype, Data Type: category
Variable: qhshort, Data Type: category
Variable: qhintd, Data Type: int8
Variable: qhintm, Data Type: int8
Variable: qhinty, Data Type: int16
Variable: qhintnum, Data Type: int16
Variable: qhresult, Data Type: category
Variable: qhvisits, Data Type: int8
Variable: qhmember, Data Type: float64
Variable: qhwomen, Data Type: float64
Variable: qhdwomen, Data Type: float64
Variable: qhresp, Data Type: float64
Variable: qhlangq, Data Type: category
Variable: qhlangi, Data Type: category
Variable: qhlangr, Data Type: category
Variable: qhtrans, Data Type: category
Variable: qhsuperv, Data Type: int16
Variable: qhfedit, Data Type: int16
Variable: qhkeyer, Data Type: int8
Variable: qhintc, Data Type: int16
Variable: qhintcd, Data Type: int32
Variable: qh100h, Data Type: category
Variable: qh100m, Data Type: c

<ipython-input-4-af0705fcdd66>:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for var_name, data_type in data_types1.iteritems():


In [9]:
!pip install pyreadstat
import pyreadstat

# Assuming you have a Stata dataset file named 'your_data.dta'
df, meta = pyreadstat.read_dta(abspath_curr + '/data/PK_2019_DHS/PKHH7ADT/PKHH7AFL.DTA')

# Access variable labels from the metadata
variable_labels = meta.column_labels

# Print variable labels
for i, var_label in enumerate(variable_labels):
    var_name = df.columns[i]
    print(f"Variable: {var_name}, Label: {var_label}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.1 MB/s eta 0:00:00
Variable: qhclust, Label: cluster number
Variable: qhnumber, Label: household number
Variable: qhregion, Label: region of residence
Variable: qhdist, Label: district
Variable: qhtype, Label: type of place of residence
Variable: qhshort, Label: short household questionnaire
Variable: qhintd, Label: day of household interview
Variable: qhintm, Label: month of household interview
Variable: qhinty, Label: year of household interview
Variable: qhintnum, Label: interviewer number
Variable: qhresult, Label: result of household interview
Variable: qhvisits, Label: number of household visits
Variable: qhmember, Label: total members in household
Variable: qhwomen, Label: eligible women in household
Variable: qhdwomen, Label: eligible death women in household
Variable: qhresp, Label: line number of household respondent
Variable: qhlangq, Label: language of questionnaire
Variable: qhlangi, Label: language of interview
V

In [12]:
dhs_data1["qh103"]

0         in own dwelling
1                     NaN
2                     NaN
3                     NaN
4                     NaN
               ...       
145427                NaN
145428                NaN
145429                NaN
145430                NaN
145431                NaN
Name: qh103, Length: 145432, dtype: category
Categories (3, object): ['in own dwelling' < 'in own yard/plot' < 'elsewhere']

In [5]:
# Initialize variables
dhs_data1['HV009'] = 0
dhs_data1['HV012'] = 0
dhs_data1['HV013'] = 0
dhs_data1['domestic'] = 0
dhs_data1['house'] = 0
dhs_data1['land'] = 0

# Loop through the DataFrame
for index, row in dhs_data1.iterrows():
    if row['qhresult'] != 1:
        continue

    # Calculate HV012 and HV013
    hv012 = sum([1 for i in range(1, row['soccurs_qhsec01'] + 1) if row[f'qh05_{i}'] == 1])
    hv013 = sum([1 for i in range(1, row['soccurs_qhsec01'] + 1) if row[f'qh06_{i}'] == 1])

    # Update HV009
    hv009 = row['soccurs_qhsec01']

    # Update domestic, house, and land
    if row['q811'] == 913 and row['qh03_qline'] == 12:
        dhs_data1.at[index, 'domestic'] = 1
    if row['q806'] == 913 and row['qh03_qline'] == 12:
       dhs_data1.at[index, 'domestic'] = 1
    if 1 <= row['q826a'] <= 3:
       dhs_data1.at[index, 'house'] = 1
    if 1 <= row['q826b'] <= 3:
       dhs_data1.at[index, 'land'] = 1

    # Update the DataFrame with the calculated values
    dhs_data1.at[index, 'HV009'] = hv009
    dhs_data1.at[index, 'HV012'] = hv012
    dhs_data1.at[index, 'HV013'] = hv013

# Export the modified DataFrame to a new CSV file
dhs_data1.to_csv('output.csv', index=False)

In [6]:
print(dhs_data1.columns)

Index(['qhclust', 'qhnumber', 'qhregion', 'qhdist', 'qhtype', 'qhshort',
       'qhintd', 'qhintm', 'qhinty', 'qhintnum',
       ...
       'qsh122j', 'qsh142', 'qsh143', 'qsh144', 'HV009', 'HV012', 'HV013',
       'domestic', 'house', 'land'],
      dtype='object', length=156)


In [7]:

# Create new columns with default values
dhs_data1['h2oires'] = 0
dhs_data1['h2oyrd'] = 0
dhs_data1['h2opub'] = 0
dhs_data1['h2obwell'] = 0
dhs_data1['h2ipwell'] = 0
dhs_data1['h2iowell'] = 0
dhs_data1['h2opspg'] = 0
dhs_data1['h2ouspg'] = 0
dhs_data1['h2orain'] = 0
dhs_data1['h2otruck'] = 0
dhs_data1['h2ocart'] = 0
dhs_data1['h2osurf'] = 0
dhs_data1['h2obot'] = 0
dhs_data1['h2ooth'] = 0

# Set values based on conditions
dhs_data1.loc[dhs_data1['qh102'] == 11, 'h2oires'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 12, 'h2oyrd'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 13, 'h2opub'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 21, 'h2obwell'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 31, 'h2ipwell'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 32, 'h2iowell'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 41, 'h2opspg'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 42, 'h2ouspg'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 51, 'h2orain'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 61, 'h2otruck'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 71, 'h2ocart'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 81, 'h2osurf'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 91, 'h2obot'] = 1
dhs_data1.loc[dhs_data1['qh102'] == 96, 'h2ooth'] = 1






# Create new columns with default values
dhs_data1['flushs'] = 0
dhs_data1['flusht'] = 0
dhs_data1['flushp'] = 0
dhs_data1['flushe'] = 0
dhs_data1['latvip'] = 0
dhs_data1['latpits'] = 0
dhs_data1['latpit'] = 0
dhs_data1['latcomp'] = 0
dhs_data1['latpail'] = 0
dhs_data1['lathang'] = 0
dhs_data1['latbush'] = 0
dhs_data1['latoth'] = 0
dhs_data1['latshare'] = 0

# Set values based on conditions
dhs_data1.loc[dhs_data1['qh107'] == 11, 'flushs'] = 1
dhs_data1.loc[dhs_data1['qh107'] == 12, 'flusht'] = 1
dhs_data1.loc[dhs_data1['qh107'] == 13, 'flushp'] = 1
dhs_data1.loc[dhs_data1['qh107'].isin([14, 15]), 'flushe'] = 1
dhs_data1.loc[dhs_data1['qh107'] == 21, 'latvip'] = 1
dhs_data1.loc[dhs_data1['qh107'] == 22, 'latpits'] = 1
dhs_data1.loc[dhs_data1['qh107'] == 23, 'latpit'] = 1
dhs_data1.loc[dhs_data1['qh107'] == 31, 'latcomp'] = 1
dhs_data1.loc[dhs_data1['qh107'] == 41, 'latpail'] = 1
dhs_data1.loc[dhs_data1['qh107'] == 51, 'lathang'] = 1
dhs_data1.loc[dhs_data1['qh107'] == 61, 'latbush'] = 1
dhs_data1.loc[dhs_data1['qh107'] == 96, 'latoth'] = 1
dhs_data1.loc[dhs_data1['qh108'] == 1, 'latshare'] = 1

# Create new columns with default values
dhs_data1['dirtfloo'] = 0
dhs_data1['woodfloo'] = 0
dhs_data1['prqfloo'] = 0
dhs_data1['vinlfloo'] = 0
dhs_data1['tilefloo'] = 0
dhs_data1['cemtfloo'] = 0
dhs_data1['rugfloo'] = 0
dhs_data1['othfloo'] = 0

# Set values based on conditions
dhs_data1.loc[dhs_data1['qh114'].isin([11, 12]), 'dirtfloo'] = 1
dhs_data1.loc[dhs_data1['qh114'].isin([21, 22]), 'woodfloo'] = 1
dhs_data1.loc[dhs_data1['qh114'] == 31, 'prqfloo'] = 1
dhs_data1.loc[dhs_data1['qh114'] == 32, 'vinlfloo'] = 1
dhs_data1.loc[dhs_data1['qh114'] == 33, 'tilefloo'] = 1
dhs_data1.loc[dhs_data1['qh114'] == 34, 'cemtfloo'] = 1
dhs_data1.loc[dhs_data1['qh114'] == 35, 'rugfloo'] = 1
dhs_data1.loc[dhs_data1['qh114'] == 96, 'othfloo'] = 1

# Create new columns with default values
dhs_data1['nowall'] = 0
dhs_data1['natwall'] = 0
dhs_data1['mudwall'] = 0
dhs_data1['mdstwall'] = 0
dhs_data1['adobwall'] = 0
dhs_data1['plywall'] = 0
dhs_data1['cardwall'] = 0
dhs_data1['rwoodwall'] = 0
dhs_data1['cmtwall'] = 0
dhs_data1['stonwall'] = 0
dhs_data1['brkwall'] = 0
dhs_data1['cmtbwall'] = 0
dhs_data1['cadobwall'] = 0
dhs_data1['woodwall'] = 0
dhs_data1['othwall'] = 0

# Set values based on conditions
dhs_data1.loc[dhs_data1['qh116'] == 11, 'nowall'] = 1
dhs_data1.loc[dhs_data1['qh116'].isin([12, 13]), 'natwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 21, 'mudwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 22, 'mdstwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 23, 'adobwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 24, 'plywall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 25, 'cardwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 26, 'rwoodwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 31, 'cmtwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 32, 'stonwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 33, 'brkwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 34, 'cmtbwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 35, 'cadobwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 36, 'woodwall'] = 1
dhs_data1.loc[dhs_data1['qh116'] == 96, 'othwall'] = 1

# Create new columns with default values
dhs_data1['noroof'] = 0
dhs_data1['natroof'] = 0
dhs_data1['matroof'] = 0
dhs_data1['bambroof'] = 0
dhs_data1['wproof'] = 0
dhs_data1['cardroof'] = 0
dhs_data1['metroof'] = 0
dhs_data1['woodroof'] = 0
dhs_data1['asbroof'] = 0
dhs_data1['cmtroof'] = 0
dhs_data1['shngroof'] = 0
dhs_data1['othroof'] = 0

# Set values based on conditions
dhs_data1.loc[dhs_data1['qh115'] == 11, 'noroof'] = 1
dhs_data1.loc[dhs_data1['qh115'] == 12, 'natroof'] = 1
dhs_data1.loc[dhs_data1['qh115'] == 21, 'matroof'] = 1
dhs_data1.loc[dhs_data1['qh115'] == 22, 'bambroof'] = 1
dhs_data1.loc[dhs_data1['qh115'] == 23, 'wproof'] = 1
dhs_data1.loc[dhs_data1['qh115'] == 24, 'cardroof'] = 1
dhs_data1.loc[dhs_data1['qh115'] == 31, 'metroof'] = 1
dhs_data1.loc[dhs_data1['qh115'] == 32, 'woodroof'] = 1
dhs_data1.loc[dhs_data1['qh115'] == 33, 'asbroof'] = 1
dhs_data1.loc[dhs_data1['qh115'] == 34, 'cmtroof'] = 1
dhs_data1.loc[dhs_data1['qh115'] == 35, 'shngroof'] = 1
dhs_data1.loc[dhs_data1['qh115'] == 96, 'othroof'] = 1

# Create new columns with default values
dhs_data1['cookelec'] = 0
dhs_data1['cooklpg'] = 0
dhs_data1['cookgas'] = 0
dhs_data1['cookbio'] = 0
dhs_data1['cookkero'] = 0
dhs_data1['cookchar'] = 0
dhs_data1['cookwood'] = 0
dhs_data1['cookstraw'] = 0
dhs_data1['cookcrop'] = 0
dhs_data1['cookdung'] = 0
dhs_data1['cooknone'] = 0
dhs_data1['cookoth'] = 0

# Set values based on conditions
dhs_data1.loc[dhs_data1['qh111'] == 1, 'cookelec'] = 1
dhs_data1.loc[dhs_data1['qh111'] == 2, 'cooklpg'] = 1
dhs_data1.loc[dhs_data1['qh111'] == 3, 'cookgas'] = 1
dhs_data1.loc[dhs_data1['qh111'] == 4, 'cookbio'] = 1
dhs_data1.loc[dhs_data1['qh111'] == 5, 'cookkero'] = 1
dhs_data1.loc[dhs_data1['qh111'] == 6, 'cookchar'] = 1
dhs_data1.loc[dhs_data1['qh111'] == 7, 'cookwood'] = 1
dhs_data1.loc[dhs_data1['qh111'] == 8, 'cookstraw'] = 1
dhs_data1.loc[dhs_data1['qh111'] == 9, 'cookcrop'] = 1
dhs_data1.loc[dhs_data1['qh111'] == 10, 'cookdung'] = 1
dhs_data1.loc[dhs_data1['qh111'] == 95, 'cooknone'] = 1
dhs_data1.loc[dhs_data1['qh111'] == 96, 'cookoth'] = 1

# Create new columns with default values for lighting
dhs_data1['eleclgt'] = 0
dhs_data1['sunlgt'] = 0
dhs_data1['gaslgt'] = 0
dhs_data1['hurrlgt'] = 0
dhs_data1['preslgt'] = 0
dhs_data1['wicklgt'] = 0
dhs_data1['candlgt'] = 0
dhs_data1['woodlgt'] = 0
dhs_data1['othlgt'] = 0

# Set values based on conditions for lighting
dhs_data1.loc[dhs_data1['qh106'] == 1, 'eleclgt'] = 1
dhs_data1.loc[dhs_data1['qh106'] == 2, 'sunlgt'] = 1
dhs_data1.loc[dhs_data1['qh106'] == 3, 'gaslgt'] = 1
dhs_data1.loc[dhs_data1['qh106'] == 4, 'hurrlgt'] = 1
dhs_data1.loc[dhs_data1['qh106'] == 5, 'preslgt'] = 1
dhs_data1.loc[dhs_data1['qh106'] == 6, 'wicklgt'] = 1
dhs_data1.loc[dhs_data1['qh106'] == 8, 'candlgt'] = 1
dhs_data1.loc[dhs_data1['qh106'] == 7, 'woodlgt'] = 1
dhs_data1.loc[dhs_data1['qh106'] == 96, 'othlgt'] = 1


# Export the modified DataFrame to a new CSV file
dhs_data1.to_csv('output.csv', index=False)


In [13]:
print(dhs_data1.columns)

Index(['qhclust', 'qhnumber', 'qhregion', 'qhdist', 'qhtype', 'qhshort',
       'qhintd', 'qhintm', 'qhinty', 'qhintnum',
       ...
       'cookoth', 'eleclgt', 'sunlgt', 'gaslgt', 'hurrlgt', 'preslgt',
       'wicklgt', 'candlgt', 'woodlgt', 'othlgt'],
      dtype='object', length=239)


In [20]:
from pandas.core.tools.datetimes import unique
test= dhs_data1["qh101"]
test1= unique(test)
print(test1)

['tube well or borehole', NaN, 'piped - public tap / standpipe', 'piped - into dwelling', 'piped - to yard/plot', ..., 'tanker truck', 'filtration plant', 'rainwater', 'other', 'cart with small tank']
Length: 17
Categories (16, object): ['piped - into dwelling' < 'piped - to yard/plot' < 'piped - to neighbor' <
                          'piped - public tap / standpipe' ... 'cart with small tank' <
                          'surface water (river/dam/lake/pond/stream/can... < 'bottled water' < 'other']
